In [2]:
import pandas as pd
import re
import os

# ============================================================
# 🧭 Y06_안전비상벨_TM 수치화 전처리 파이프라인
# - 컬럼 한글화
# - 시·구·동 파싱 및 결측 보완
# - 결측 제거 전후 비교
# - 구/동별 설치 수 집계
# - 서울시 25개 자치구 텍스트 요약
# - 최종 저장
# ============================================================

# ===== 0) 경로 설정 =====
input_file  = "/Users/mac/Documents/SORA_Project/data/raw/Y06_안전비상벨_TM.csv"
output_dir  = "/Users/mac/Documents/SORA_Project/data/preprocessing"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "Y06_전처리_안전비상벨_TM.csv")

# ===== 1) 파일 로드 및 기본 정보 =====
df = pd.read_csv(input_file, sep="\t", encoding="utf-8-sig")
print(f"\n✅ [1단계] 파일 로드 완료: {df.shape[0]:,}행 × {df.shape[1]}열")
df.info()
print("\n📌 원본 컬럼명:", df.columns.tolist())


✅ [1단계] 파일 로드 완료: 1,000행 × 39열
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 39 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   _OBJT_ID    1000 non-null   object 
 1   FCLTY_TY    908 non-null    object 
 2   MNG_INST    908 non-null    object 
 3   INS_PURPOS  908 non-null    object 
 4   INS_TYPE    908 non-null    object 
 5   INS_DETAIL  908 non-null    object 
 6   RN_ADRES    908 non-null    object 
 7   ADRES       908 non-null    object 
 8   LAT         908 non-null    float64
 9   LON         908 non-null    float64
 10  LNK_TYPE    908 non-null    object 
 11  FLAG_POL_L  908 non-null    object 
 12  FLAG_SEC_L  908 non-null    object 
 13  FLAG_MNG_L  908 non-null    object 
 14  ADDITION    908 non-null    object 
 15  INS_YEAR    908 non-null    float64
 16  LAST_INSPD  908 non-null    float64
 17  LAST_INSPT  908 non-null    object 
 18  MNG_TEL     908 non-null    object 
 

In [3]:
df.head(10)

,_OBJT_ID,FCLTY_TY,MNG_INST,INS_PURPOS,INS_TYPE,INS_DETAIL,RN_ADRES,ADRES,LAT,LON,...,CLSS,PNU,주소구분,표준신주소,표준구주소,우편번호,행정동코드,행정동명,법정동코드,법정동명
0,1,비상벨,종로구청,약자보호,화장실,인왕산도시자연공원(청운지구 서시정),-,서울특별시 종로구 청운동 7-3,37.591042,126.966257,...,정좌표,1.111010e+18,구주소,NaN,서울특별시 종로구 청운동 7-3,NaN,1.111052e+09,청운효자동,1.111010e+09,청운동
1,2,비상벨,종로구청,방범용,공원,인왕산도시자연공원,-,서울특별시 종로구 청운동 7-3,37.591042,126.966257,...,정좌표,1.111010e+18,구주소,NaN,서울특별시 종로구 청운동 7-3,NaN,1.111052e+09,청운효자동,1.111010e+09,청운동
2,3,비상벨,종로구청,약자보호,화장실,인왕산도시자연공원(청운지구),-,서울특별시 종로구 청운동 7-27,37.589514,126.965510,...,정좌표,1.111010e+18,구주소,NaN,서울특별시 종로구 청운동 7-27,NaN,NaN,NaN,1.111010e+09,청운동
3,4,비상벨,종로구청,약자보호,화장실,창의문화장실,-,서울특별시 종로구 청운동 산 1-1,37.591324,126.971784,...,정좌표,1.111010e+18,구주소,서울특별시 종로구 창의문로 42,서울특별시 종로구 청운동 산1-1,3048.0,1.111052e+09,청운효자동,1.111010e+09,청운동
4,5,비상벨,종로구청,약자보호,화장실,창의문화장실,-,서울특별시 종로구 청운동 산 1-1,37.591324,126.971784,...,정좌표,1.111010e+18,구주소,서울특별시 종로구 창의문로 42,서울특별시 종로구 청운동 산1-1,3048.0,1.111052e+09,청운효자동,1.111010e+09,청운동
5,6,비상벨,종로구청,약자보호,화장실,무궁화동산,-,서울특별시 종로구 궁정동 55-3,37.584386,126.972402,...,정좌표,1.111010e+18,구주소,NaN,서울특별시 종로구 궁정동 55-3,NaN,1.111052e+09,청운효자동,1.111010e+09,궁정동
6,7,비상벨,종로구청 청소행정과,방범용,화장실,누상동체육시설 공중화장실,-,서울특별시 종로구 누상동 산 1-3,37.580046,126.964202,...,정좌표,1.111011e+18,구주소,서울특별시 종로구 옥인6길 26-17,서울특별시 종로구 누상동 산1-3,3038.0,1.111052e+09,청운효자동,1.111011e+09,누상동
7,8,비상벨,종로구청,약자보호,화장실,인왕산도시자연공원(누상지구),-,서울특별시 종로구 누상동 산 1-38,37.578974,126.963164,...,정좌표,1.111011e+18,구주소,NaN,서울특별시 종로구 누상동 산1-38,NaN,1.111052e+09,청운효자동,1.111011e+09,누상동
8,9,비상벨,종로구청,방범용,공원,수성동계곡,-,서울특별시 종로구 옥인동 185-4,37.582070,126.963764,...,정좌표,1.111011e+18,구주소,NaN,서울특별시 종로구 옥인동 185-4,NaN,NaN,NaN,1.111011e+09,옥인동
9,10,비상벨,종로구청 청소행정과,방범용,화장실,인왕산수목원약수터 공중화장실,-,서울특별시 종로구 옥인동 179-1,37.583030,126.963573,...,정좌표,1.111011e+18,구주소,NaN,서울특별시 종로구 옥인동 179-1,3034.0,1.111052e+09,청운효자동,1.111011e+09,옥인동


In [4]:
# ===== 2) 컬럼명 한글화 =====
col_map = {
    "OBJT_ID": "일련번호", "FCLTY_TY": "유형명", "MNG_INST": "관리기관명",
    "INS_PURPOS": "설치목적", "INS_TYPE": "설치장소유형", "INS_DETAIL": "설치위치",
    "RN_ADRES": "도로명주소", "ADRES": "지번주소", "LAT": "위도", "LON": "경도",
    "LNK_TYPE": "연계방식", "FLAG_POL_L": "경찰연계유무", "FLAG_SEC_L": "경비업체연계유무",
    "FLAG_MNG_L": "관리사무소연계유무", "ADDITION": "부가기능", "INS_YEAR": "안전비상벨설치연도",
    "LAST_INSPD": "최종점검일자", "LAST_INSPT": "최종점검결과구분", "MNG_TEL": "관리기관전화번호",
    "FLAG_SERVI": "연계유무", "CTPRVN_CD": "시도코드", "SGG_CD": "시군구코드", "EMD_CD": "읍면동코드",
    "X": "X좌표", "Y": "Y좌표", "DATA_TY": "데이터기준일자"
}
df.rename(columns={c: col_map[c] for c in df.columns if c in col_map}, inplace=True)

print("\n✅ [2단계] 컬럼명 한글 변환 완료")
print("📌 변환 후 컬럼명:", df.columns.tolist())

# ===== 3) 시·구·동 파싱 및 결측 보완 =====
def parse_and_fill_address(row):
    # 1) 행정동명 또는 법정동명 기준 파싱
    base = None
    if '행정동명' in df.columns and pd.notna(row.get('행정동명')):
        base = row['행정동명']
    elif '법정동명' in df.columns and pd.notna(row.get('법정동명')):
        base = row['법정동명']

    city, gu, dong = None, None, None
    if base:
        parts = str(base).strip().split()
        city = "서울특별시"
        gu = parts[1] if len(parts) > 1 else None
        dong = parts[2] if len(parts) > 2 else None

    # 2) 도로명주소 / 지번주소로 보완
    for col in ['도로명주소', '지번주소']:
        if (gu is None or dong is None) and pd.notna(row.get(col)):
            addr_parts = str(row[col]).strip().split()
            if gu is None and len(addr_parts) > 1:
                gu = addr_parts[1]
            if dong is None and len(addr_parts) > 2:
                dong = addr_parts[2]

    # 3) 시 결측값 통일
    if city is None or str(city).strip() in ['', 'nan', 'None']:
        city = '서울특별시'

    return pd.Series([city, gu, dong])

df[['시', '구', '동']] = df.apply(parse_and_fill_address, axis=1)

print("\n✅ [3단계] 시·구·동 파싱 및 결측 보완 완료")
print(df[['시', '구', '동']].head(10))
print(f"📊 시 결측치 수: {df['시'].isnull().sum()}")
print(f"📊 구 결측치 수: {df['구'].isnull().sum()}")
print(f"📊 동 결측치 수: {df['동'].isnull().sum()}")
print("\n📌 시 값 분포:\n", df['시'].value_counts().head())


✅ [2단계] 컬럼명 한글 변환 완료
📌 변환 후 컬럼명: ['_OBJT_ID', '유형명', '관리기관명', '설치목적', '설치장소유형', '설치위치', '도로명주소', '지번주소', '위도', '경도', '연계방식', '경찰연계유무', '경비업체연계유무', '관리사무소연계유무', '부가기능', '안전비상벨설치연도', '최종점검일자', '최종점검결과구분', '관리기관전화번호', '연계유무', '시도코드', '시군구코드', '읍면동코드', 'X좌표', 'Y좌표', '데이터기준일자', '입력주소', 'X.1', 'Y.1', 'CLSS', 'PNU', '주소구분', '표준신주소', '표준구주소', '우편번호', '행정동코드', '행정동명', '법정동코드', '법정동명']

✅ [3단계] 시·구·동 파싱 및 결측 보완 완료
       시    구    동
0  서울특별시  종로구  청운동
1  서울특별시  종로구  청운동
2  서울특별시  종로구  청운동
3  서울특별시  종로구  청운동
4  서울특별시  종로구  청운동
5  서울특별시  종로구  궁정동
6  서울특별시  종로구  누상동
7  서울특별시  종로구  누상동
8  서울특별시  종로구  옥인동
9  서울특별시  종로구  옥인동
📊 시 결측치 수: 0
📊 구 결측치 수: 92
📊 동 결측치 수: 92

📌 시 값 분포:
 시
서울특별시    1000
Name: count, dtype: int64


In [5]:
# ===== 4) 결측치 제거 전후 비교 =====
before_count = len(df)
df_cleaned = df.dropna(subset=['구', '동']).reset_index(drop=True)
after_count = len(df_cleaned)
removed_count = before_count - after_count

print("\n✅ [4단계] 결측치 제거 완료")
print(f"📊 원본 데이터 수: {before_count:,} 행")
print(f"📊 결측 제거 후 데이터 수: {after_count:,} 행")
print(f"📉 제거된 행 수: {removed_count:,} 행")

# ===== 5) 지역구별 / 동별 설치 수 집계 =====
df_cleaned['지역구별 비상벨 설치수'] = df_cleaned.groupby('구')['동'].transform('size')
df_cleaned['동별 비상벨 설치수'] = df_cleaned.groupby(['구', '동'])['동'].transform('size')

print("\n✅ [5단계] 수치 집계 완료")
print(df_cleaned[['시', '구', '동', '지역구별 비상벨 설치수', '동별 비상벨 설치수']].head(10))



✅ [4단계] 결측치 제거 완료
📊 원본 데이터 수: 1,000 행
📊 결측 제거 후 데이터 수: 908 행
📉 제거된 행 수: 92 행

✅ [5단계] 수치 집계 완료
       시    구    동  지역구별 비상벨 설치수  동별 비상벨 설치수
0  서울특별시  종로구  청운동            47           5
1  서울특별시  종로구  청운동            47           5
2  서울특별시  종로구  청운동            47           5
3  서울특별시  종로구  청운동            47           5
4  서울특별시  종로구  청운동            47           5
5  서울특별시  종로구  궁정동            47           1
6  서울특별시  종로구  누상동            47           2
7  서울특별시  종로구  누상동            47           2
8  서울특별시  종로구  옥인동            47           2
9  서울특별시  종로구  옥인동            47           2


In [6]:
# ===== 6) 최종 테이블 생성 (서울시 25개 자치구 포함) =====

# 6-1. 구별 설치 수 집계
df_gu_count = (
    df_cleaned.groupby('구')
    .size()
    .reset_index(name='지역구별 비상벨 설치수')
    .sort_values(by='지역구별 비상벨 설치수', ascending=False)
)

# 6-2. 동별 설치 수 집계
df_dong_count = (
    df_cleaned.groupby(['구', '동'])
    .size()
    .reset_index(name='동별 비상벨 설치수')
)

# 6-3. 컬럼 정리
df_result = df_dong_count.rename(columns={'구': '지역구별', '동': '동별'})
df_result = df_result[['지역구별', '동별', '동별 비상벨 설치수']]
df_result = df_result.sort_values(
    by=['지역구별', '동별 비상벨 설치수'],
    ascending=[True, False]
).reset_index(drop=True)

# 📌 구와 동 오름차순 정렬 추가
df_result = df_result.sort_values(by=['지역구별', '동별'], ascending=[True, True]).reset_index(drop=True)

# 6-4. 📌 서울시 25개 자치구 포함 (없는 구는 0개)
seoul_districts = [
    "종로구","중구","용산구","성동구","광진구","동대문구","중랑구","성북구","강북구","도봉구",
    "노원구","은평구","서대문구","마포구","양천구","강서구","구로구","금천구","영등포구",
    "동작구","관악구","서초구","강남구","송파구","강동구"
]
df_all_gu = pd.DataFrame({"구": seoul_districts})
df_all_gu = df_all_gu.merge(df_gu_count, on="구", how="left")
df_all_gu['지역구별 비상벨 설치수'] = df_all_gu['지역구별 비상벨 설치수'].fillna(0).astype(int)

# 6-5. 텍스트 요약 생성
gu_summary_text = ", ".join(
    [f"{row['구']} {row['지역구별 비상벨 설치수']}개" for _, row in df_all_gu.iterrows()]
)

# 6-6. 출력
print("\n✅ [6단계] 최종 테이블 생성 완료")
print(f"📊 지역구별 총 설치수 요약: {gu_summary_text}")
print("\n📌 동별 수치화 데이터 샘플:")
print(df_result.head(20))



✅ [6단계] 최종 테이블 생성 완료
📊 지역구별 총 설치수 요약: 종로구 47개, 중구 599개, 용산구 262개, 성동구 0개, 광진구 0개, 동대문구 0개, 중랑구 0개, 성북구 0개, 강북구 0개, 도봉구 0개, 노원구 0개, 은평구 0개, 서대문구 0개, 마포구 0개, 양천구 0개, 강서구 0개, 구로구 0개, 금천구 0개, 영등포구 0개, 동작구 0개, 관악구 0개, 서초구 0개, 강남구 0개, 송파구 0개, 강동구 0개

📌 동별 수치화 데이터 샘플:
   지역구별         동별  동별 비상벨 설치수
0   용산구        갈월동          10
1   용산구        남영동           4
2   용산구        동자동           7
3   용산구      두텁바위로           4
4   용산구   두텁바위로13길           1
5   용산구    두텁바위로1길           1
6   용산구   두텁바위로1나길           2
7   용산구  두텁바위로38가길           1
8   용산구   두텁바위로69길           1
9   용산구        백범로           1
10  용산구        서계동          26
11  용산구     소월로20길           1
12  용산구     소월로26길           1
13  용산구     소월로2가길           2
14  용산구     소월로30길           1
15  용산구        신창동           7
16  용산구        신흥로           3
17  용산구    신흥로11나길           1
18  용산구     신흥로16길           1
19  용산구     신흥로20길           2


In [7]:
# ===== 7) 결과 저장 =====
df_result.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"\n💾 [7단계] 'Y06_전처리_안전비상벨_TM.csv' 저장 완료")
print(f"📊 최종 행 수: {len(df_result):,} 행")


💾 [7단계] 'Y06_전처리_안전비상벨_TM.csv' 저장 완료
📊 최종 행 수: 232 행
